In [1]:
import os
import re
import h5py

import numpy as np

from tqdm import tqdm

1. Sample preparation
* Generate .root file
* Select the events and save jet constitutent information to .h5 file
    - Sample/from_root_to_h5.py
* Generate mixed sample
* Data augmentation
    - optional
    - Sample/physical_augmentation_h5.ipynb
* From .h5 file generate jet image and save in .npy file
    - Sample/from_h5_to_npy.py
2. CNN Training

# Sample preparation

## From root to h5

In [2]:
root_path = '../../Software/pythia8307/HVmodel/test_100k.root'
output_path = './HVmodel/data/new/signal.h5'
sample_type = 1

cmd = f'python from_root_to_h5.py {root_path} {output_path} {sample_type} &'
print(cmd)

root_path = '../../Software/pythia8307/HVmodel/test_100k-2.root'
output_path = './HVmodel/data/new/signal-test.h5'
sample_type = 1

cmd = f'python from_root_to_h5.py {root_path} {output_path} {sample_type} &'
print(cmd)

root_path = './ppjj/Events/run_03/tag_1_delphes_events.root'
output_path = './HVmodel/data/new/background_03.h5'
sample_type = 0

cmd = f'python from_root_to_h5.py {root_path} {output_path} {sample_type} &'
print(cmd)

root_path = './ppjj/Events/run_04/tag_1_delphes_events.root'
output_path = './HVmodel/data/new/background-test.h5'
sample_type = 0

cmd = f'python from_root_to_h5.py {root_path} {output_path} {sample_type} &'
print(cmd)

root_path = './ppjj/Events/run_05/tag_1_delphes_events.root'
output_path = './HVmodel/data/new/background_05.h5'
sample_type = 0

cmd = f'python from_root_to_h5.py {root_path} {output_path} {sample_type} &'
print(cmd)

python from_root_to_h5.py ../../Software/pythia8307/HVmodel/test_100k.root ./HVmodel/data/new/signal.h5 1 &
python from_root_to_h5.py ../../Software/pythia8307/HVmodel/test_100k-2.root ./HVmodel/data/new/signal-test.h5 1 &
python from_root_to_h5.py ./ppjj/Events/run_03/tag_1_delphes_events.root ./HVmodel/data/new/background_03.h5 0 &
python from_root_to_h5.py ./ppjj/Events/run_04/tag_1_delphes_events.root ./HVmodel/data/new/background-test.h5 0 &
python from_root_to_h5.py ./ppjj/Events/run_05/tag_1_delphes_events.root ./HVmodel/data/new/background_05.h5 0 &


## Generate mixed sample

### Pre-selection 

In [8]:
import ROOT

ROOT.gROOT.ProcessLine('.include /usr/local/Delphes-3.4.2/')
ROOT.gROOT.ProcessLine('.include /usr/local/Delphes-3.4.2/external/')
ROOT.gInterpreter.Declare('#include "/usr/local/Delphes-3.4.2/classes/DelphesClasses.h"')
ROOT.gInterpreter.Declare('#include "/usr/local/Delphes-3.4.2/external/ExRootAnalysis/ExRootTreeReader.h"')
ROOT.gInterpreter.Declare('#include "/usr/local/Delphes-3.4.2/external/ExRootAnalysis/ExRootConfReader.h"')
ROOT.gInterpreter.Declare('#include "/usr/local/Delphes-3.4.2/external/ExRootAnalysis/ExRootTask.h"')
ROOT.gSystem.Load("/usr/local/Delphes-3.4.2/install/lib/libDelphes")

0

In [ ]:
def Mjets(*arg):
    # arg: list of jets
    # return: invariant mass of jets
    e_tot, px_tot, py_tot, pz_tot = 0, 0, 0, 0
    
    for jet in arg:
        pt, eta, phi, m = jet[0], jet[1], jet[2], jet[3]
        
        px, py, pz = pt*np.cos(phi), pt*np.sin(phi), pt*np.sinh(eta)
        e = np.sqrt(m**2 + px**2 + py**2 + pz**2)
        
        px_tot += px
        py_tot += py
        pz_tot += pz
        e_tot += e
    
    return np.sqrt(e_tot**2 - px_tot**2 - py_tot**2 - pz_tot**2)

In [ ]:
def HV_selection(tree):
    # Hidden Valley model selection
    # 1. 2 jets
    # 2. pT > 750 GeV
    # 3. |eta| < 2.0

    n_event_count = 0
    n_jet_count = 0
    jet_pt_count = 0
    jet_eta_count = 0
    mjjs = []

    pt = [[],[]]

    for event_id, event in tqdm(enumerate(tree)):
        n_event_count += 1

        if event.Jet_size < 2:
            continue
        n_jet_count += 1

        pt[0].append(event.Jet[0].PT)
        pt[1].append(event.Jet[1].PT)
        if event.Jet[1].PT < 750:
            continue
        jet_pt_count += 1

        if abs(event.Jet[0].Eta) > 2.0 or abs(event.Jet[1].Eta) > 2.0:
            continue
        jet_eta_count += 1

        jets = [[event.Jet[i].PT, event.Jet[i].Eta, event.Jet[i].Phi, event.Jet[i].Mass] for i in range(2)]
        mjj = Mjets(*jets)
        mjjs.append(mjj)

        if mjj < 4300 or mjj > 5900:
            continue


    mjjs = np.array(mjjs)
    SR_count = ((mjjs > 4700) & (mjjs < 5500)).sum()
    SB_count = (((mjjs > 4300) & (mjjs < 4700)) | ((mjjs > 5500) & (mjjs < 5900))).sum()

    cutflow_number = {
        'Total': n_event_count,
        'n jet cut': n_jet_count,
        'jet pt cut': jet_pt_count,
        'jet eta cut': jet_eta_count,
        'Signal region': SR_count,
        'Sideband region': SB_count,
    }

    results = {
        'mjj': mjjs,
        'pt': np.array(pt),
        'cutflow_number': cutflow_number,
    }

    return results

In [ ]:
root_file = '../../Software/pythia8307/HVmodel/test_100k.root'
f = ROOT.TFile(root_file)
tree_s = f.Get("Delphes")

results_s = HV_selection(tree_s)

100000it [02:41, 620.00it/s]


In [ ]:
root_file = './ppjj/Events/run_02/tag_1_delphes_events.root'
f = ROOT.TFile(root_file)
tree_b = f.Get("Delphes")

results_b = HV_selection(tree_b)

100000it [02:18, 720.06it/s]
Warning in <TStreamerInfo::BuildCheck>: 
   The StreamerInfo for version 2 of class GenParticle read from the file ./ppjj/Events/run_02/tag_1_delphes_events.root
   has a different checksum than the previously loaded StreamerInfo.
   Reading objects of type GenParticle from the file ./ppjj/Events/run_02/tag_1_delphes_events.root 
   (and potentially other files) might not work correctly.
   Most likely the version number of the class was not properly
   updated [See ClassDef(GenParticle,2)].
Warning in <TStreamerInfo::CompareContent>: The following data member of
the on-file layout version 2 of class 'GenParticle' differs from 
the in-memory layout version 2:
   float T; //number
vs
   float CtgTheta; //number
Warning in <TStreamerInfo::CompareContent>: The following data member of
the on-file layout version 2 of class 'GenParticle' differs from 
the in-memory layout version 2:
   float X; //number
vs
   float D0; //number
Warning in <TStreamerInfo::Compare

In [ ]:
np.save('./HVmodel/data/selection_results_s.npy', results_s)
np.save('./HVmodel/data/selection_results_b.npy', results_b)

### Compute sample size

In [3]:
def get_info(path):
    # path: run path
    name = os.path.split(path)[1]

    with open(os.path.join(path, f'{name}_tag_1_banner.txt')) as f:
        for line in f.readlines():
                
            #  Integrated weight (pb)  :       0.020257
            match = re.match('#  Integrated weight \(pb\)  : +(\d+\.\d+)', line)
            if match:
                # unit: fb
                cross_section = float(match.group(1)) * 1000
            # #  Number of Events        :       100000
            match = re.match('#  Number of Events        :       (\d+)', line)
            if match:
                # unit: fb
                nevent = int(match.group(1))
    
    return cross_section, nevent

def get_dataset_keys(f):
    keys = []
    f.visit(lambda key : keys.append(key) if isinstance(f[key], h5py.Dataset) else None)
    return keys

def create_dataset(f, nevent, MAX_JETS):

    f.create_dataset('J1/MASK', (nevent, MAX_JETS), maxshape=(None, MAX_JETS), dtype='|b1')
    f.create_dataset('J1/pt', (nevent, MAX_JETS), maxshape=(None, MAX_JETS), dtype='<f4')
    f.create_dataset('J1/eta', (nevent, MAX_JETS), maxshape=(None, MAX_JETS), dtype='<f4')
    f.create_dataset('J1/phi', (nevent, MAX_JETS), maxshape=(None, MAX_JETS), dtype='<f4')

    f.create_dataset('J2/MASK', (nevent, MAX_JETS), maxshape=(None, MAX_JETS), dtype='|b1')
    f.create_dataset('J2/pt', (nevent, MAX_JETS), maxshape=(None, MAX_JETS), dtype='<f4')
    f.create_dataset('J2/eta', (nevent, MAX_JETS), maxshape=(None, MAX_JETS), dtype='<f4')
    f.create_dataset('J2/phi', (nevent, MAX_JETS), maxshape=(None, MAX_JETS), dtype='<f4')

    f.create_dataset('EVENT/Mjj', (nevent,), maxshape=(None,), dtype='<f4')
    f.create_dataset('EVENT/signal', (nevent,), maxshape=(None,), dtype='<i8')

In [4]:
results_s = np.load('./Sample/HVmodel/data/selection_results_s.npy', allow_pickle=True).item()
results_b = np.load('./Sample/HVmodel/data/selection_results_b.npy', allow_pickle=True).item()

In [5]:
# Total cross section and number of events
xection, tot_event = get_info('./Sample/ppjj/Events/run_03')
print(xection, tot_event)

# cross section in signal region and sideband region
cross_section_SR = results_b['cutflow_number']['Signal region'] / results_b['cutflow_number']['Total'] * xection
cross_section_SB = results_b['cutflow_number']['Sideband region'] / results_b['cutflow_number']['Total'] * xection
print(f'Background cross section, SR: {cross_section_SR:.2f} fb, SB: {cross_section_SB:.2f} fb')

# number of background events in signal region and sideband region
L = 139 * 1
n_SR_B = cross_section_SR * L
n_SB_B = cross_section_SB * L

print(f'Background sample size: SR: {n_SR_B:.1f}, SB: {n_SB_B:.1f}')

sensitivity = 1
n_SR_S = sensitivity * np.sqrt(n_SR_B)
n_SB_S = n_SR_S * results_s['cutflow_number']['Sideband region'] / results_s['cutflow_number']['Signal region']
print(n_SR_S, n_SB_S)

6837.392481 1000000
Background cross section, SR: 136.13 fb, SB: 211.28 fb
Background sample size: SR: 18922.4, SB: 29367.3
137.55877041193227 39.606895448991104


### Make Training sample

In [6]:
def create_mix_sample_from_numbers(sig_path, bkg_path, n_sig_1, n_sig_2, n_bkg_1, n_bkg_2, output_path):
    # n_sig_1: number of signal events in mixing sample 1 (Signal region)
    # n_sig_2: number of signal events in mixing sample 2 (Sideband region)

    nevent = n_sig_1 + n_sig_2 + n_bkg_1 + n_bkg_2
    with h5py.File(output_path, 'w') as f_out:
        MAX_JETS = 300 
        create_dataset(f_out, nevent, MAX_JETS)

        keys = get_dataset_keys(f_out)
        with h5py.File(sig_path, 'r') as f_sig, h5py.File(bkg_path, 'r') as f_bkg:  
            mjj_s = f_sig['EVENT/Mjj'][:]
            mjj_b = f_bkg['EVENT/Mjj'][:]
            SR_range_s = (mjj_s > 4700) & (mjj_s < 5500)
            SB_range_s = ((mjj_s > 4300) & (mjj_s < 4700)) | ((mjj_s > 5500) & (mjj_s < 5900))
            SR_range_b = (mjj_b > 4700) & (mjj_b < 5500)
            SB_range_b = ((mjj_b > 4300) & (mjj_b < 4700)) | ((mjj_b > 5500) & (mjj_b < 5900))

            for key in keys:
                f_out[key][:n_sig_1] = f_sig[key][:][SR_range_s][:n_sig_1]
                f_out[key][n_sig_1:n_sig_1+n_bkg_1] = f_bkg[key][:][SR_range_b][:n_bkg_1]
                f_out[key][n_sig_1+n_bkg_1:n_sig_1+n_bkg_1+n_sig_2] = f_sig[key][:][SB_range_s][:n_sig_2]
                f_out[key][n_sig_1+n_bkg_1+n_sig_2:] = f_bkg[key][:][SB_range_b][:n_bkg_2]

        f_out['EVENT/signal'][:n_sig_1+n_bkg_1] = 1
        f_out['EVENT/signal'][n_sig_1+n_bkg_1:] = 0

In [ ]:
sig_path = './Sample/HVmodel/data/new/signal.h5'
bkg_path = './Sample/HVmodel/data/new/background_03.h5'
n_sig_1, n_sig_2, n_bkg_1, n_bkg_2 = int(n_SR_S), int(n_SB_S), int(n_SR_B), int(n_SB_B)
output_path = './Sample/HVmodel/data/new/mix_sample.h5'
create_mix_sample_from_numbers(sig_path, bkg_path, n_sig_1, n_sig_2, n_bkg_1, n_bkg_2, output_path)

In [21]:
sig_path = './Sample/HVmodel/data/new/signal.h5'
bkg_path = './Sample/HVmodel/data/new/background_03.h5'

for i in range(11):
    sensitivity = i
    n_SR_S = sensitivity * np.sqrt(n_SR_B)
    n_SB_S = n_SR_S * results_s['cutflow_number']['Sideband region'] / results_s['cutflow_number']['Signal region']

    n_sig_1, n_sig_2, n_bkg_1, n_bkg_2 = int(n_SR_S), int(n_SB_S), int(n_SR_B), int(n_SB_B)
    output_path = f'./Sample/HVmodel/data/new/mix_sample_{i:.1f}.h5'
    create_mix_sample_from_numbers(sig_path, bkg_path, n_sig_1, n_sig_2, n_bkg_1, n_bkg_2, output_path)

In [23]:
sig_path = './Sample/HVmodel/data/new/signal.h5'
bkg_path = './Sample/HVmodel/data/new/background_03_merged.h5'

for i in range(11):
    sensitivity = i
    n_SR_S = sensitivity * np.sqrt(n_SR_B)
    n_SB_S = n_SR_S * results_s['cutflow_number']['Sideband region'] / results_s['cutflow_number']['Signal region']

    n_sig_1, n_sig_2, n_bkg_1, n_bkg_2 = int(n_SR_S), int(n_SB_S), int(n_SR_B), int(n_SB_B)
    output_path = f'./Sample/HVmodel/data/new/mix_sample_{i:.1f}_x2.h5'
    create_mix_sample_from_numbers(sig_path, bkg_path, n_sig_1, n_sig_2, n_bkg_1, n_bkg_2, output_path)

### Make Testing sample

In [24]:
sig_path, bkg_path  = './Sample/HVmodel/data/new/signal-test.h5', './Sample/HVmodel/data/new/background-test.h5'
n_sig_1, n_bkg_1 = 10000, 10000
nevent = n_sig_1 + n_bkg_1

with h5py.File('./Sample/HVmodel/data/new/mix_sample_testing.h5', 'w') as f_out:
    
    MAX_JETS = 300 
    create_dataset(f_out, nevent, MAX_JETS)

    keys = get_dataset_keys(f_out)
    with h5py.File(sig_path, 'r') as f_sig, h5py.File(bkg_path, 'r') as f_bkg:  
        mjj_s = f_sig['EVENT/Mjj'][:]
        mjj_b = f_bkg['EVENT/Mjj'][:]
        SR_range_s = (mjj_s > 4700) & (mjj_s < 5500)
        SR_range_b = (mjj_b > 4700) & (mjj_b < 5500)

        for key in keys:
            f_out[key][:n_sig_1] = f_sig[key][:][SR_range_s][:n_sig_1]
            f_out[key][n_sig_1:] = f_bkg[key][:][SR_range_b][:n_bkg_1]

    f_out['EVENT/signal'][:n_sig_1] = 1
    f_out['EVENT/signal'][n_sig_1:] = 0

## Data augmentation

Sample/physical_augmentation_h5.ipynb

## From h5 to npy

In [7]:
h5_path = f'./HVmodel/data/new/mix_sample_testing.h5'
output_path = f'./HVmodel/data/new/mix_sample_testing.npy'

cmd = f'python from_h5_to_npy.py {h5_path} {output_path} &'
print(cmd)

python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_testing.h5 ./HVmodel/data/new/mix_sample_testing.npy &


In [8]:
for i in range(11):
    h5_path = f'./HVmodel/data/new/mix_sample_{i:.1f}.h5'
    output_path = f'./HVmodel/data/new/mix_sample_{i:.1f}.npy'

    cmd = f'python from_h5_to_npy.py {h5_path} {output_path} &'
    print(cmd)

python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_0.0.h5 ./HVmodel/data/new/mix_sample_0.0.npy &
python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_1.0.h5 ./HVmodel/data/new/mix_sample_1.0.npy &
python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_2.0.h5 ./HVmodel/data/new/mix_sample_2.0.npy &
python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_3.0.h5 ./HVmodel/data/new/mix_sample_3.0.npy &
python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_4.0.h5 ./HVmodel/data/new/mix_sample_4.0.npy &
python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_5.0.h5 ./HVmodel/data/new/mix_sample_5.0.npy &
python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_6.0.h5 ./HVmodel/data/new/mix_sample_6.0.npy &
python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_7.0.h5 ./HVmodel/data/new/mix_sample_7.0.npy &
python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_8.0.h5 ./HVmodel/data/new/mix_sample_8.0.npy &
python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_9.0.h5 ./HVmodel/data/new/m

In [9]:
for i in range(11):
    h5_path = f'./HVmodel/data/new/mix_sample_{i:.1f}_aug_1.h5'
    output_path = f'./HVmodel/data/new/mix_sample_{i:.1f}_aug_1.npy'

    cmd = f'python from_h5_to_npy.py {h5_path} {output_path} &'
    print(cmd)

python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_0.0_aug_1.h5 ./HVmodel/data/new/mix_sample_0.0_aug_1.npy &
python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_1.0_aug_1.h5 ./HVmodel/data/new/mix_sample_1.0_aug_1.npy &
python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_2.0_aug_1.h5 ./HVmodel/data/new/mix_sample_2.0_aug_1.npy &
python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_3.0_aug_1.h5 ./HVmodel/data/new/mix_sample_3.0_aug_1.npy &
python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_4.0_aug_1.h5 ./HVmodel/data/new/mix_sample_4.0_aug_1.npy &
python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_5.0_aug_1.h5 ./HVmodel/data/new/mix_sample_5.0_aug_1.npy &
python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_6.0_aug_1.h5 ./HVmodel/data/new/mix_sample_6.0_aug_1.npy &
python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_7.0_aug_1.h5 ./HVmodel/data/new/mix_sample_7.0_aug_1.npy &
python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_8.0_aug_1.h5 ./HVmodel/data/new/m

In [10]:
for i in range(11):
    h5_path = f'./HVmodel/data/new/mix_sample_{i:.1f}_x2.h5'
    output_path = f'./HVmodel/data/new/mix_sample_{i:.1f}_x2.npy'

    cmd = f'python from_h5_to_npy.py {h5_path} {output_path} &'
    print(cmd)

python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_0.0_x2.h5 ./HVmodel/data/new/mix_sample_0.0_x2.npy &
python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_1.0_x2.h5 ./HVmodel/data/new/mix_sample_1.0_x2.npy &
python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_2.0_x2.h5 ./HVmodel/data/new/mix_sample_2.0_x2.npy &
python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_3.0_x2.h5 ./HVmodel/data/new/mix_sample_3.0_x2.npy &
python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_4.0_x2.h5 ./HVmodel/data/new/mix_sample_4.0_x2.npy &
python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_5.0_x2.h5 ./HVmodel/data/new/mix_sample_5.0_x2.npy &
python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_6.0_x2.h5 ./HVmodel/data/new/mix_sample_6.0_x2.npy &
python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_7.0_x2.h5 ./HVmodel/data/new/mix_sample_7.0_x2.npy &
python from_h5_to_npy.py ./HVmodel/data/new/mix_sample_8.0_x2.h5 ./HVmodel/data/new/mix_sample_8.0_x2.npy &
python from_h5_to_npy.py ./H

# CNN Training

In [11]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/new/mix_sample_{i:.1f}.npy'
    model_name = f'SB_{i:.1f}_new'
    sample_type = f'Sensitivity: {i:.1f}'
    cmd = f'python train_CNN.py {train_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_0.0.npy SB_0.0_new "Sensitivity: 0.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_1.0.npy SB_1.0_new "Sensitivity: 1.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_2.0.npy SB_2.0_new "Sensitivity: 2.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_3.0.npy SB_3.0_new "Sensitivity: 3.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_4.0.npy SB_4.0_new "Sensitivity: 4.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_5.0.npy SB_5.0_new "Sensitivity: 5.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_6.0.npy SB_6.0_new "Sensitivity: 6.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_7.0.npy SB_7.0_new "Sensitivity: 7.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_8.0.npy SB_8.0_new "Sensitivity: 8.0"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_9.0.npy SB_9.0_new "Sensitivity: 9.0"
python train_CNN.py ../Sample/HVmodel/da

In [12]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/new/mix_sample_{i:.1f}_aug_1.npy'
    model_name = f'SB_{i:.1f}_aug_1_new'
    sample_type = f'Sensitivity: {i:.1f}, Augmentation: 1'
    cmd = f'python train_CNN.py {train_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_0.0_aug_1.npy SB_0.0_aug_1_new "Sensitivity: 0.0, Augmentation: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_1.0_aug_1.npy SB_1.0_aug_1_new "Sensitivity: 1.0, Augmentation: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_2.0_aug_1.npy SB_2.0_aug_1_new "Sensitivity: 2.0, Augmentation: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_3.0_aug_1.npy SB_3.0_aug_1_new "Sensitivity: 3.0, Augmentation: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_4.0_aug_1.npy SB_4.0_aug_1_new "Sensitivity: 4.0, Augmentation: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_5.0_aug_1.npy SB_5.0_aug_1_new "Sensitivity: 5.0, Augmentation: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_6.0_aug_1.npy SB_6.0_aug_1_new "Sensitivity: 6.0, Augmentation: 1"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_7.0_aug_1.npy SB_7.0_aug_1_new "Sensitivity: 7.0, Augmentation: 1"


In [13]:
for i in range(11):
    train_file = f'../Sample/HVmodel/data/new/mix_sample_{i:.1f}_x2.npy'
    model_name = f'SB_{i:.1f}_x2_new'
    sample_type = f'Sensitivity: {i:.1f}, Luminosity: x2'
    cmd = f'python train_CNN.py {train_file} {model_name} "{sample_type}"'
    print(cmd)

python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_0.0_x2.npy SB_0.0_x2_new "Sensitivity: 0.0, Luminosity: x2"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_1.0_x2.npy SB_1.0_x2_new "Sensitivity: 1.0, Luminosity: x2"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_2.0_x2.npy SB_2.0_x2_new "Sensitivity: 2.0, Luminosity: x2"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_3.0_x2.npy SB_3.0_x2_new "Sensitivity: 3.0, Luminosity: x2"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_4.0_x2.npy SB_4.0_x2_new "Sensitivity: 4.0, Luminosity: x2"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_5.0_x2.npy SB_5.0_x2_new "Sensitivity: 5.0, Luminosity: x2"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_6.0_x2.npy SB_6.0_x2_new "Sensitivity: 6.0, Luminosity: x2"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sample_7.0_x2.npy SB_7.0_x2_new "Sensitivity: 7.0, Luminosity: x2"
python train_CNN.py ../Sample/HVmodel/data/new/mix_sampl